<a href="https://colab.research.google.com/github/ajdillhoff/CSE6363/blob/main/imdb-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import torchtext
import spacy
import torch
import torchtext.transforms as T

ModuleNotFoundError: ignored

In [57]:
train_dataset, test_dataset = torchtext.datasets.IMDB()

Build vocabulary

In [58]:
tokenizer = torchtext.data.utils.get_tokenizer("spacy", language="en_core_web_sm")
train_datapipe = train_dataset.map(lambda x: tokenizer(x[1]))
v = torchtext.vocab.build_vocab_from_iterator(train_datapipe, specials=["<unk>"])
print(v.get_itos()[:2])

['<unk>', 'the']


In [59]:
test_datapipe = test_dataset.map(lambda x: (v(tokenizer(x[1])), x[0]))

In [60]:
next(iter(test_datapipe))

RuntimeError: ignored